In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# "https://colab.research.google.com/github/TWCkaijin/GDGC-Gemini-bootcamp/blob/main/function_calling.ipynb"

# Function Calling 與 Gemini API & Python SDK 介紹

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/TWCkaijin/GDGC-Gemini-bootcamp/blob/main/function_calling/function_calling.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FTWCkaijin%2FGDGC-Gemini-bootcamp%2Fmain%2FFunction_Calling%2ffunction_calling.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  
</table>



|Original Author(s) | [Kristopher Overholt](https://github.com/koverholt) [Holt Skinner](https://github.com/holtskinner) |

概述
YouTube 影片：AI + 你的程式碼：函式呼叫（Function Calling）

<a href="https://www.youtube.com/watch?v=NbAGbXr4DME&list=PLIivdWyY5sqLvGdVLJZh2EMax97_T-OIB" target="_blank"> <img src="https://img.youtube.com/vi/NbAGbXr4DME/maxresdefault.jpg" alt="AI + 你的程式碼：函式呼叫" width="500"> </a>

</br>

## Gemini
Gemini 是 Google DeepMind 開發的一系列生成式 AI 模型，專為多模態（multimodal）應用場景設計。


</br>

## 從 Gemini 呼叫函式
函式呼叫（Function Calling） 讓開發者可以在程式碼中描述一個函式，然後將該描述傳遞給語言模型進行請求。模型的回應會包含符合該描述的函式名稱以及應該使用的參數。

</br>

## 為什麼要使用函式呼叫？
想像一下，如果你請某人記錄重要資訊，但沒有提供表格或任何格式指引，對方可能會寫出一篇流暢的文章，但如果你需要從中提取特定的姓名、日期或數字，將會非常費力！同樣地，若沒有函式呼叫，想要從生成式文本模型獲得一致的結構化數據會是一大挑戰。你可能得不斷要求模型輸出 JSON 格式，但結果往往不穩定且令人沮喪。

這正是 Gemini 函式呼叫 的優勢所在。與其期待從自由格式的文本回應中拼湊出所需資訊，不如直接定義清楚的函式，並指定具體的參數和資料型別。這些函式定義相當於結構化的指引，能讓 Gemini 以可預測且可用的方式輸出結果。這樣一來，你就不需要再從文字回應中手動解析重要資訊了！

可以把它想像成教 Gemini 如何與你的應用程式對話。
需要從資料庫檢索資訊？定義一個 search_db 函式，並指定搜尋條件作為參數。
想要與天氣 API 整合？建立一個 get_weather 函式，並讓它接收地點作為輸入。
函式呼叫能夠橋接自然語言與結構化數據，讓 AI 更輕鬆地與外部系統互動！

## 任務目標
在本教學中，你將學習如何在 Vertex AI 中使用 Gemini API，並透過 Vertex AI SDK for Python 來使用 Gemini 2.0 Flash (gemini-2.0-flash) 模型進行函式呼叫（Function Calling）。

你將完成以下任務：

- 安裝 _Google Gen AI SDK for Python_
- 在 _Vertex AI_ 中使用 _Gemini API_ 與 _Gemini_ 模型互動：
- 在聊天會話（chat session）中使用 函式呼叫，回答使用者關於 Google Store 產品的問題
- 使用 _Function Calling_ 透過 地圖 API 進行地址地理編碼（geocoding）
- 使用 _Function Calling_ 在 原始日誌數據（raw logging data） 中進行實體擷取（entity extraction）

付費資源
本教學將使用 Google Cloud 中的 計費 功能：

- Vertex AI
請參閱 [Vertex AI](https://cloud.google.com/vertex-ai/pricing)價格 以了解詳細計費資訊，並使用 [費用估算](https://cloud.google.com/products/calculator/) 根據你的預計使用量估算成本。

## Getting Started


### 安裝 Google Gen AI SDK 套件


In [1]:
%pip install --upgrade  google-genai

  Using cached google_genai-1.5.0-py3-none-any.whl.metadata (29 kB)
Using cached google_genai-1.5.0-py3-none-any.whl (142 kB)
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.3.0
    Uninstalling google-genai-1.3.0:
      Successfully uninstalled google-genai-1.3.0
Note: you may need to restart the kernel to use updated packages.


### 重啟Colab執行個體

您剛剛安裝了一個套件，為確保他正確載入，我們將重啟執行個體

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

""" app = IPython.Application.instance()
app.kernel.do_shutdown(True) """

<div class="alert alert-block alert-warning">
<b>⚠️ 您的執行個體將會重啟，您會在左下角看到警示訊息 ⚠️</b>
</div>


### 驗證Colab環境 (Colab & local only)

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
else: 
    try: 
        from google.oauth2.service_account import Credentials
        SERVICE_ACCOUNT_FILE = 'secret.json'  # Path to your JSON file
        SCOPES = ['https://www.googleapis.com/auth/cloud-platform']  # IMPORTANT: Add the explicit scope
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    except ImportError:
        print('If you are running this in a colab enterprise, please ignore this message.')
        print("Import error, please install the google-auth library check if you secret file are in the correct path")


### 設定 Google Cloud Platform 專案


要開始使用 Vertex AI，你需要擁有一個 現有的 Google Cloud 專案，並[啟用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

你可以參考[設定專案與開發環境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)來了解更多詳細資訊。

In [3]:
import os

PROJECT_ID = "side-projcet-placeholder"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION, credentials=creds)

## 範例程式碼

### 選擇模型

想要了解更多關於Vertax AI 的 AI 模型 和 APIs, see [Google Models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models) and [Model Garden](https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models).

In [4]:
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}

### 載入套件


In [62]:
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool, Schema
import requests

### 聊天範例：在聊天會話中使用函式呼叫回答使用者關於 Google Store 的問題


在這個範例中，我們將使用 Gemini 的函式呼叫（Function Calling） 來建立一個聊天機器人，該機器人可以回答使用者關於 Google Store 產品的問題。

這樣的應用可以讓 AI 透過結構化的函式呼叫，而不是單純生成自由格式的文字，來提供更準確和一致的資訊。

In [63]:
get_product_info = FunctionDeclaration(
    name="get_product_info",
    description="Get the stock amount and identifier for a given product",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product_name": {"type": "STRING", "description": "Product name"}
        },
    },
)

get_store_location = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "OBJECT",
        "properties": {"location": {"type": "STRING", "description": "Location"}},
    },
)

place_order = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product": {"type": "STRING", "description": "Product name"},
            "address": {"type": "STRING", "description": "Shipping address"},
        },
    },
)
    



請注意，函式參數需按照 [OpenAPI JSON Schema](https://spec.openapis.org/oas/v3.0.3#schemawr).
 格式以Python 字典（dictionary） 形式指定。

以下是定義工具（tool）的方法，讓 Gemini 模型 可以從 3 個函式 中進行選擇：


In [64]:
retail_tool = Tool(
    function_declarations=[
        get_product_info,
        get_store_location,
        place_order,
    ],
)

現在，你可以在多輪對話（multi-turn chat session）中初始化 Gemini 模型，並啟用函式呼叫（Function Calling）。

在初始化聊天會話時，可以透過 `tools` 參數一次性指定可用的函式，這樣在後續的請求中就不需要每次重新傳遞這些函式設定。

In [65]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        temperature=0,
        tools=[retail_tool],
    ),
)

**注意事項**：
temperature 參數控制生成回應的隨機性：

較低的 temperature（如 0）：適合需要 **確定性（deterministic）** 的函式，例如傳遞固定格式的參數。
較高的 temperature：適合允許更具創意或多樣性參數值的函式，例如需要較自由輸入的應用場景。


當 temperature = 0 時，模型的輸出大多是確定性的，但仍可能有些許變化。

In [66]:
prompt = """
Do you have the Pixel 9 in stock?
"""

response = chat.send_message(prompt)

response.function_calls[0]

FunctionCall(id=None, args={'product_name': 'Pixel 9'}, name='get_product_info')

Gemini API 的回應包含一個結構化的資料物件，其中包括 Gemini 從可用函式中選擇的函式名稱以及對應的參數。

由於本教學的重點是提取函式參數並生成函式呼叫，因此你將使用模擬數據（mock data）來回傳合成回應給 Gemini 模型，而不是直接向 API 伺服器發送請求。（不用擔心！稍後的範例中，我們會進行實際的 API 呼叫。）

In [67]:
#在這裡，你可以使用你偏好的方法來發送 API 請求並獲取回應。
#在本範例中，我們將使用**模擬數據（synthetic data）**來模擬來自外部 API 的回應內容。

api_response = {"sku": "GA04834-US", "in_stock": "yes"}

在實際應用中，你會使用適當的客戶端函式庫或 REST API，對外部系統或資料庫執行函式呼叫。

現在，你可以將來自（模擬的）API 請求的回應傳遞給 Gemini 模型，並生成最終的使用者回應。

In [68]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
display(Markdown(response.text))

Yes, we have the Pixel 9 in stock.


接下來，使用者可能會詢問在哪裡可以從附近的商店購買其他手機：

In [69]:
prompt = """
What about the Pixel 9 Pro XL? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.function_calls

[FunctionCall(id=None, args={'product_name': 'Pixel 9 Pro XL'}, name='get_product_info'),
 FunctionCall(id=None, args={'location': 'Mountain View, CA'}, name='get_store_location')]

同樣地，你會收到一個包含結構化資料的回應，但這次請注意——這次不只是一個函式呼叫，而是兩個！

Gemini 模型 判斷需要同時呼叫 `get_product_info` 和 `get_store_location` 這兩個函式。

回頭看看幾個步驟前的對話提示，你會發現使用者詢問的不只是產品資訊，還有店鋪位置。

當多個函式被呼叫時
當定義了多個函式（或當模型預測需要對同一函式執行多次呼叫）時，Gemini 模型 可能會在同一回合內返回連續或並行的函式呼叫。

這些行為都是預料之內的，因為 Gemini 模型 會根據 **運行時推理（runtime prediction）** 來決定：

1. 應該呼叫哪些函式
2. 函式的執行順序（如果有相依性，則會依序執行）
3. 哪些函式可以並行執行，以便快速獲取足夠的資訊來生成自然語言回應

</br>

不用擔心！你可以重複相同的步驟，模擬 API 回應，來構造來自外部 API 的合成回應（synthetic payloads）。

In [70]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

product_info_api_response = {"sku": "GA08475-US", "in_stock": "yes"}
store_location_api_response = {
    "store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"
}

同樣地，你可以將來自（模擬的）API 請求的回應傳遞回 Gemini 模型。

In [71]:
response = chat.send_message(
    [
        Part.from_function_response(
            name="get_product_info",
            response={
                "content": product_info_api_response,
            },
        ),
        Part.from_function_response(
            name="get_store_location",
            response={
                "content": store_location_api_response,
            },
        ),
    ]
)
display(Markdown(response.text))

Yes, we have the Pixel 9 Pro XL in stock. The store located at 2000 N Shoreline Blvd, Mountain View, CA 94043, US is the closest store to you.


### 做得很好！

在單次對話回合內，Gemini 模型 連續請求了 2 個函式呼叫，然後才返回自然語言摘要。

在實際應用中，如果你需要查詢庫存系統，然後再向店鋪位置資料庫、客戶管理系統或文件存儲系統發送 API 請求，這種模式將非常有用。

最後，使用者可能會請求訂購手機並將其配送到指定地址：

In [72]:
prompt = """
I'd like to order a Pixel 9 Pro XL and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.function_calls

[FunctionCall(id=None, args={'address': '1155 Borregas Ave, Sunnyvale, CA 94089', 'product': 'Pixel 9 Pro XL'}, name='place_order')]

太棒了！Gemini 模型 成功提取了使用者選擇的產品和他們的地址。現在，你可以呼叫 API 來完成訂單處理：

In [73]:
# This is where you would make an API request to return the status of their order.
# Use synthetic data to simulate a response payload from an external API.

order_api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

並將來自外部 API 呼叫的負載（payload）傳遞給 Gemini API，這樣 Gemini API 就會生成一個自然語言摘要，並回傳給最終使用者。

In [74]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": order_api_response,
        },
    ),
)
display(Markdown(response.text))

OK. I've placed an order for a Pixel 9 Pro XL to be shipped to 1155 Borregas Ave, Sunnyvale, CA 94089. The order number is 12345 and it should arrive in 2 days.


恭喜你！

你成功地與 Gemini 模型 進行了多輪對話，使用函式呼叫、處理負載，並生成包含來自外部系統資訊的自然語言摘要

### 延伸-地址範例：使用自動函式呼叫（Automatic Function Calling）來透過地圖 API 進行地址地理編碼（geocode）

在這個範例中，你將定義一個接收多個參數的函式。然後，你將使用 Gemini API 的自動函式呼叫來執行實際的 API 呼叫，將地址轉換為緯度和經度座標。

首先，撰寫一個 Python 函式：

In [75]:
def get_location(
    amenity: str | None = None,
    street: str | None = None,
    city: str | None = None,
    county: str | None = None,
    state: str | None = None,
    country: str | None = None,
    postalcode: str | None = None,
) -> list[dict]:
    """
    Get latitude and longitude for a given location.

    Args:
        amenity (str | None): Amenity or Point of interest.
        street (str | None): Street name.
        city (str | None): City name.
        county (str | None): County name.
        state (str | None): State name.
        country (str | None): Country name.
        postalcode (str | None): Postal code.

    Returns:
        list[dict]: A list of dictionaries with the latitude and longitude of the given location.
                    Returns an empty list if the location cannot be determined.
    """
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "amenity": amenity,
        "street": street,
        "city": city,
        "county": county,
        "state": state,
        "country": country,
        "postalcode": postalcode,
        "format": "json",
    }
    
    # Filter out None values from parameters
    params = {k: v for k, v in params.items() if v is not None}

    try:
        response = requests.get(base_url, params=params, headers={"User-Agent": "none"})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching location data: {e}")
        return []

在這個範例中，你會要求 Gemini 模型 將地址的組件提取到結構化資料物件中的特定欄位。這些欄位會被傳遞到你所定義的函式，然後回傳結果給 Gemini，以生成自然語言回應。

例如，發送一個包含地址的提示：

In [76]:
prompt = """
I want to get the coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

The coordinates for 1600 Amphitheatre Pkwy, Mountain View, CA 94043 are: latitude 37.42248575, longitude -122.08558456613565.



做得很好！你成功地定義了一個函式，並讓 Gemini 模型 用來從提示中提取相關的參數。接著，你進行了實際的 API 呼叫，獲取了指定位置的座標。

在這裡，我們使用了 [OpenStreetMap Nominatim API](https://nominatim.openstreetmap.org/ui/search.html) 來進行地址的地理編碼，以便讓本教學的步驟數量保持在合理範圍。如果你需要處理大量地址或地理位置數據，也可以使用 [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding)，或者任何提供 API 的地圖服務！

## Conclusions

你已完成探索 Google Gen AI Python SDK 的 `function calling` feature

深入探索 [更多資訊](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling).

---

## 實作 一

我們現在來點簡單的實作

- 定義一個加法的*function*
- 以 *Automatic function calling* 的方式提供資料
- 向AI提供你的prompt讓他決定要不要執行你的code!


### 1. import 必需的套件

In [ ]:
from google.genai.types import GenerateContentConfig
MODEL_ID = "gemini-2.0-flash-001"

### 2. 定義間單的加法

In [ ]:
def add_fn(a: float, b: float) -> float:
    return a+b

def subtract_fn(a: float, b: float) -> float:
    return a-b

### 3. 使用`generate_content`方法，給定參數(模型, 輸入, 設定(工具, 創意度))

In [ ]:
prompt = """
請問36-52等於多少？
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=["????"], temperature=2),
)
print(response.text)

## 實作 二: 高雄地區公車資訊API
來簡單的呼叫及整理API資料吧~

首先，我們先定義一些資訊
(例如: 台灣公車公開資料API網址、其他套件等等)



In [6]:
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool, Schema
import requests
import random
import time
from pprint import pprint
import json

app_id = 'B123245005-18b880b2-9380-44b8'
app_key = 'e55d12ce-e5d2-40e3-9adc-c7272cf46704'

auth_url="https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
url = "https://tdx.transportdata.tw/api/basic/v2/Bus/RealTimeByFrequency/City/Kaohsiung?%24top=10&health=false&%24format=JSON"

auth_response = None

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


##### 再來，定義一些關於我們要的實際操作內容

1. 取得API的憑證金鑰
2. 使用金鑰向API請求資料
3. 取得票劵資料訊息 (模擬)
4. 訂購票劵 (模擬)
5. 查詢位置的GeoCode

In [7]:
def get_auth():
    """
    這段程式可以取的TDX服務帳號中的授權token，取得後將有約4小時的有效期限
    如果還沒有授權token，將無法取得資料。
    """
    try:
        global app_id, app_key, auth_response
        content_type = 'application/x-www-form-urlencoded'
        grant_type = 'client_credentials'
        header = {
            'content-type' : content_type,
            'grant_type' : grant_type,
            'client_id' : app_id,
            'client_secret' : app_key
        }
        auth_response = requests.post(auth_url, header)
    except :
        raise Exception("Error fetching auth data")

    
def get_bus_info():

    """
    這段程式可以為使用者取得高雄市的公車資訊，包含公車路線名稱、公車位置、公車狀態
    若未持有授權token，將無法取得資料。
    """
    try:
        global app_id, app_key, auth_response
        if auth_response is None:
            get_auth()
        auth_JSON = json.loads(auth_response.text)
        access_token = auth_JSON.get('access_token')
        header = {
            'authorization': 'Bearer ' + access_token,
            'Accept-Encoding': 'gzip'
        }
        data_response = requests.get(url, headers=header)
        if data_response.status_code == 200:
            return [{"routeName":station['RouteName'], "currentposition":station['BusPosition'], "OnDuty":station["DutyStatus"]} for station in data_response.json()]
            return data_response.json()
        else:
            raise Exception("Bus data api error")
    except :
        raise Exception("Error fetching bus data")
    



def get_ticket_info(location: dict, time: str) -> dict:
    """
    這段程式將模擬使用者傳入[位置、時間]嘗試取得票券的情況，並回傳是否有票券的資訊
    """

    result = random.choice([{f"{time}": "yes"}, {f"{time}": "no"}])
    return result



def reserve_ticket(location: dict, time: str) -> dict:
    """
    這段程式將模擬使用者傳入[位置、時間]嘗試預約票券的情況，並回傳是否預約成功的資訊
    """
    check = get_ticket_info(location, time)
    if check[time] == "yes":
        print(f"Getting tickets")
        return {"status": "success", "message": "Ticket reserved"}
    elif (check[time] == "no"):
        return {"status": "fail", "message": "No ticket available"}
    return {"status": "fail", "message": "Unknown error"}

def get_location(
    street: str | None = None,
) -> list[dict]:
    
    """
    這段程式可以取得使用者傳入的地址(或地標)的經緯度資訊
    """
    
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "street": street,
        "format": "json",
    }
    
    # Filter out None values from parameters
    params = {k: v for k, v in params.items() if v is not None}

    try:
        response = requests.get(base_url, params=params, headers={"User-Agent": "none"})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching location data: {e}")
        return []

#### 接著，將所有的function 定義成 Function Declaration

In [8]:
get_auth_tool = FunctionDeclaration(
    name="get_auth",
    description="Get the authentication token",
)

get_bus_info_tool = FunctionDeclaration(
    name="get_bus_info",
    description="Get the bus information",
)

get_ticket_info_tool = FunctionDeclaration(
    name="get_ticket_info",
    description="Get the ticket information",
    parameters={
        "type": "OBJECT",
        "properties": {
            "location": {"type": "OBJECT", "description": "Location"},
            "time": {"type": "STRING", "description": "Time"},
        },
    },
)

reserve_ticket_tool = FunctionDeclaration(
    name="reserve_ticket",
    description="Reserve a ticket",
    parameters={
        "type": "OBJECT",
        "properties": {
            "location": {"type": "OBJECT", "description": "Location"},
            "time": {"type": "STRING", "description": "Time"},
        },
    },
)

get_location_tool = FunctionDeclaration(
    name="get_location",
    description="Get the location of the given address",
    parameters={
        "type": "OBJECT",
        "properties": {
            "street": {"type": "STRING", "description": "Street name"},
        },
    },
)



#### 然後把所有的函式本體和Function Declaration 物件包裝並載入至模型。隨後新增System instruction 至 LLM中。

In [9]:
function_list = {"get_bus_info": get_bus_info, "get_ticket_info": get_ticket_info, "reserve_ticket": reserve_ticket, "get_auth": get_auth, "get_location": get_location}
function_declaration = [get_bus_info_tool, get_ticket_info_tool, reserve_ticket_tool, get_auth_tool, get_location_tool]

MODEL_ID = "gemini-2.0-flash-001"
retail_tool = Tool(function_declarations=function_declaration,)

chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        temperature=2,
        tools=[retail_tool],
        system_instruction=
            """
                You are a helpful chatbot that can help users to get the bus information and reserve the ticket.
                You can provide the bus information by calling the function `get_bus_info`.
                You can provide the ticket information by calling the function `get_ticket_info`.
                You can reserve the ticket by calling the function `reserve_ticket`.
                You can get the location by calling the function `get_location`.
                If you're facing auth issue or its your first time to call bus api, you should call the function `get_auth` first.
                You should always consider to call these functions to get the information.
                And also, you should show a user friendly message to the user, do not show the raw data to the user.
            """
    )
)

#### 接著......讓他可以重複執行!  (還更新了可以自動糾錯的方式)

In [10]:
err = False
while True:

    if not err:
        prompt = input()
        if prompt == "q":
            break
        print(f"{color.GREEN}User: {prompt}{color.END}")
        response = chat.send_message(prompt)
    
    try:
        if response.function_calls:
            function_call = response.function_calls[0]
            if function_call.name in function_list:
                print(f"Calling function: {function_call.name}")
                params = function_call.args
                result = function_list[function_call.name](**params)
                response = chat.send_message(
                    Part.from_function_response(
                        name=function_call.name,
                        response={"content": result},
                    )
                )
        print(f"{color.CYAN}AI: {response.text}{color.END}")
        err = False
    except Exception as e:
        err = True
        response = chat.send_message(
                    Part.from_function_response(
                        name=function_call.name,
                        response={"content": {"error with doing the function": str(e)}},
                    )
                )
        print(f"{color.CYAN}AI: {response.text}{color.END}")
        print(f"{color.RED}Error: {e}{color.END}")
        
    
    time.sleep(1)

User: 離中山大學最近的公車什麼時候到
AI: 請問您要從哪裡出發呢？我可以幫您查詢 ближайшие 公車的抵達時間。

User: 從中山大好
AI: 您好，請問中山大是什麼地方呢？ 方便提供更詳細的地址嗎？

User: 大學
AI: 好的，請問您要查詢哪個時間的公車呢？

User: 現在
Calling function: get_location
AI: 請問您指的是位於廣州市海珠區新港西路135號的中山大學(廣州校區南校園) 還是位於廣州市越秀區中山二路74號的中山大學(廣州校區北校園)呢？

User: 台灣的
AI: 您好， 台灣沒有找到中山大學，請問有其他名稱更精確的地點嗎？

User: 你要不要再試一次
Calling function: get_location
AI: 還是只找到廣州的中山大學，請問你說的是國立中山大學嗎？如果是的話，我可以幫你找。

User: 是
Calling function: get_location
AI: 好的，請問您要查詢什麼時間的公車呢？

User: 現在
Calling function: get_ticket_info
AI: 好的，現在有票。請問需要為您預定嗎？

User: 要
Calling function: reserve_ticket
Getting tickets
AI: 已為您預訂車票。
User: 
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/chenzhanhuang/miniforge3/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/k4/nr94nkkj46zg7xmg2myl51k00000gn/T/ipykernel_27302/367986393.py", line 9, in <module>
    response = chat.send_message(prompt)
  File "/Users/chenzhanhuang/miniforge3/lib/python3.13/site-packages/google/genai/chats.py", line 206, in send_message
    response = self._modules.generate_content(
        model=self._model,
        contents=self._curated_history + [input_content],
        config=config if config else self._config,
    )
  File "/Users/chenzhanhuang/miniforge3/lib/python3.13/site-packages/google/genai/models.py", line 5304, in generate_content
    response = self._generate_content(
        model=model, contents=contents, config=config
    )
  File "/Users/chenzhanhuang/miniforge3/lib/pytho

In [ ]:
get_bus_info()